In [7]:
# IMPORTING MODULES: UNCOMMENT BELOW:
# !pip install grpcio==1.42.0 tensorflow-serving-api==2.7.0
# !pip install keras-image-helper
import grpc
import tensorflow as tf

from keras_image_helper import create_preprocessor
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

In [6]:
# INITIALIZING TENSORFLOW-SERVING CHANNEL:
host = "localhost:8500"
channel = grpc.insecure_channel(host)
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

In [8]:
# CREATING PREPROCESSOR:
preprocessor = create_preprocessor("xception", target_size=(299, 299))

In [12]:
# PROCESSING THE URL:
url = "https://raw.githubusercontent.com/alexeygrigorev/clothing-dataset-small/master/test/pants/4aabd82c-82e1-4181-a84d-d0c6e550d26d.jpg"
X = preprocessor.from_url(url)

In [13]:
# CONVERTING NP ARRAY TO PROTOBUF:
def np_to_protobuf(data):
    return tf.make_tensor_proto(data, shape=data.shape)

In [14]:
# INITIALIZING THE PREDICTION:
pb_request = predict_pb2.PredictRequest()
pb_request.model_spec.name = "clothing-model"
pb_request.model_spec.signature_name = "serving_default"
pb_request.inputs["input_8"].CopyFrom(np_to_protobuf(X))

In [15]:
# INITIALIZING THE RESPONSE:
pb_response = stub.Predict(pb_request, timeout=15.0)
pb_response

outputs {
  key: "dense_7"
  value {
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: 1
      }
      dim {
        size: 10
      }
    }
    float_val: -1.8798643350601196
    float_val: -4.756312370300293
    float_val: -2.3595328330993652
    float_val: -1.0892643928527832
    float_val: 9.90378475189209
    float_val: -2.8261802196502686
    float_val: -3.648311138153076
    float_val: 3.2411553859710693
    float_val: -2.612095355987549
    float_val: -4.8520355224609375
  }
}
model_spec {
  name: "clothing-model"
  version {
    value: 1
  }
  signature_name: "serving_default"
}

In [17]:
# INSPECTING THE OUTPUT:
preds = pb_response.outputs["dense_7"].float_val

In [20]:
# INSPECTING THE OUTPUT:
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]
dict(zip(classes, preds))

{'dress': -1.8798643350601196,
 'hat': -4.756312370300293,
 'longsleeve': -2.3595328330993652,
 'outwear': -1.0892643928527832,
 'pants': 9.90378475189209,
 'shirt': -2.8261802196502686,
 'shoes': -3.648311138153076,
 'shorts': 3.2411553859710693,
 'skirt': -2.612095355987549,
 't-shirt': -4.8520355224609375}